In [1]:
import time
import yfinance as yf
import pandas_datareader as pdr
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from datetime import date
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle

Factors which we are going to taking
1. Index : Dow, S&P, NASDAQ, Nifty, Sgx Nifty,dollar index,
2. Growth : Us GDP, world GDP, Indian GDP, Indian PMI, us pmi
3. Infraction : US cpi, US Core CPI, India CPI, India core cpi
4. Rates : Fed intrest rates, rbi repo rate
5. Emp : US unemprate, india unemp , joblessclam us
6. household data :india avg fd rates, india bonds and debt fund returns
7. others :brent oil, leading us etf and foregin index,inr/usd, gold price, us bond yelid(10y and 2y), india vix, major mf, fii data

In [2]:
tickers = ['^NSEI','^DJI', '^GSPC', '^IXIC','BZ=F','^TNX','GC=F','INR=X','DX-Y.NYB']
df1 = pd.DataFrame()
# Loop through each ticker symbol
end = date.today()
start = date(year=end.year-16, month=end.month, day=end.day)
for ticker in tickers:
    # Download the historical data for the ticker
    data = yf.download(ticker, start=start, end=end)
    df1[ticker] = data['Close']
# Convert the index to a datetime format
df1.index = pd.to_datetime(df1.index, errors='coerce')
# Convert the index to a string format
df1.index = df1.index.strftime('%d-%m-%y')
df1 = df1.rename(columns={'^DJI': 'Dow_Jons', '^GSPC' : 'S&P_500', '^IXIC' : 'NASDAQ', 'BZ=F' : 'BRENT','^TNX' : 'BOND_10Y',
                 'GC=F' : 'Gold','INR=X' : 'Inr_Usd','^NSEI' : 'Nifty_50','DX-Y.NYB' : 'Dollar_Index'})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
df1 = df1.dropna()

In [4]:
df2 = pd.DataFrame()
FRED_INDICATORS1 = ['UNRATE','CPALTT01USM659N', 'FEDFUNDS','UNRATE','CPALTT01INM659N','INDIRLTLT01STM']
end = date.today()
start = date(year=end.year-16, month=end.month, day=end.day)
tq_fred1 = tqdm(FRED_INDICATORS1)
tq_fred1.set_description('Downloading stats from FRED:')
for indicator in tq_fred1:
  df2[indicator] = pdr.fred.FredReader(indicator, start=start, timeout=90).read()
df2.index = pd.to_datetime(df2.index, errors='coerce')
df2.index = df2.index.strftime('%d-%m-%y')
df2 = df2.rename(columns={'UNRATE': 'us_unemployment',
                 'CPALTT01USM659N': 'US_CPI','INDIRLTLT01STM' : 'bond_Return','FEDFUNDS': 'FED_RATE',
                 'CPALTT01INM659N': 'india_cpi',
              })
df2 = pd.DataFrame(df2)

In [5]:
df2.fillna(0, inplace=True)
df2 = df2.drop(df2.index[-1])
df2

,us_unemployment,US_CPI,FED_RATE,india_cpi,bond_Return
DATE,,,,,
01-06-07,4.6,2.687038,5.25,5.691057,0.00
01-07-07,4.7,2.358231,5.26,6.451613,0.00
01-08-07,4.6,1.970083,5.02,7.258065,0.00
01-09-07,4.7,2.755052,4.94,6.400000,0.00
01-10-07,4.7,3.536174,4.76,5.511811,0.00
...,...,...,...,...,...
01-10-22,3.7,7.745427,3.08,6.084868,7.46
01-11-22,3.6,7.110323,3.78,5.409706,7.36
01-12-22,3.5,6.454401,4.10,5.502392,7.29


In [6]:
def scrapping(url):
    s = Service('D:\\python projects\\stock market pricticor\\fundamandel data\\chromedriver.exe')
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    time.sleep(10)
    table_element = driver.find_element(By.XPATH,"//*[@id='js-main-container']/section[2]/div/section[2]/section[2]/div[1]" )
    table_html = table_element.get_attribute('outerHTML')
    a = pd.DataFrame()
    a = pd.read_html(table_html)[0]
    a['Date'] = pd.to_datetime(a['Date'], errors='coerce')
    a['Date'] = a['Date'].dt.strftime('%d-%m-%y')
    a = a.loc[:, ['Date', 'Price']]
    a= a.reindex(index=a.index[::-1])
    a=a.reset_index(drop=True)
    a.set_index('Date', inplace=True)
    driver.close()
    return a

In [7]:
url1 = 'https://in.investing.com/indices/india-50-futures-historical-data?cid=101810&end_date=1680852705&st_date=694204200'
url2 = 'https://in.investing.com/indices/india-vix-historical-data?end_date=1680855763&st_date=946665000'

In [8]:
a =scrapping(url1)
b = scrapping(url2)

In [9]:
a = a.rename(columns={'Price' : 'sgx'})
b = b.rename(columns={'Price' : 'indiavix'})

In [10]:
df1 = pd.concat([df1, a, b], axis=1)
df1 = df1.dropna()
df1

,Nifty_50,Dow_Jons,S&P_500,NASDAQ,BRENT,BOND_10Y,Gold,Inr_Usd,Dollar_Index,sgx,indiavix
Date,,,,,,,,,,,
04-03-08,4864.250000,12213.799805,1326.750000,2260.280029,97.519997,3.579,963.900024,40.209000,73.660004,4852.0,38.0800
05-03-08,4921.399902,12254.990234,1333.699951,2272.810059,101.639999,3.693,986.200012,40.174000,73.480003,4915.0,35.5100
07-03-08,4771.600098,11893.690430,1293.369995,2212.489990,102.379997,3.541,972.200012,40.238998,73.029999,4706.0,35.2000
10-03-08,4800.399902,11740.150391,1273.369995,2169.340088,104.160004,3.438,969.900024,40.313999,72.989998,4706.0,32.5800
11-03-08,4865.899902,12156.809570,1320.650024,2255.760010,105.250000,3.596,974.200012,40.174999,73.279999,4846.5,32.6900
...,...,...,...,...,...,...,...,...,...,...,...
29-03-23,17080.699219,32717.599609,4027.810059,11926.240234,78.279999,3.566,1966.099976,82.164299,102.639999,17200.0,13.6275
31-03-23,17359.750000,33274.148438,4109.310059,12221.910156,79.769997,3.494,1969.000000,82.100502,102.510002,17480.5,12.9350
03-04-23,17398.050781,33601.148438,4124.509766,12189.450195,84.930000,3.430,1983.900024,82.184799,102.089996,17456.5,12.5850


In [11]:
df1.columns

Index(['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y',
       'Gold', 'Inr_Usd', 'Dollar_Index', 'sgx', 'indiavix'],
      dtype='object')

In [12]:
for col in df1.columns:
    df1[f'{col}_1d_return'] = df1[col].pct_change(periods=1)
    df1[f'{col}_1d_return'] = df1[f'{col}_1d_return'].pct_change(periods=1)
    df1[f'{col}_7d_return'] = df1[col].pct_change(periods=5)
    df1[f'{col}_7d_return'] = df1[f'{col}_7d_return'].pct_change(periods=5)
    df1[f'{col}_1m_return'] = df1[col].pct_change(periods=20)
    df1[f'{col}_1m_return'] = df1[f'{col}_1m_return'].pct_change(periods=20)
    df1[f'{col}_1y_return'] = df1[col].pct_change(periods=240)
    df1[f'{col}_1y_return'] = df1[f'{col}_1y_return'].pct_change(periods=240)
df1

,Nifty_50,Dow_Jons,S&P_500,NASDAQ,BRENT,BOND_10Y,Gold,Inr_Usd,Dollar_Index,sgx,...,Dollar_Index_1m_return,Dollar_Index_1y_return,sgx_1d_return,sgx_7d_return,sgx_1m_return,sgx_1y_return,indiavix_1d_return,indiavix_7d_return,indiavix_1m_return,indiavix_1y_return
Date,,,,,,,,,,,,,,,,,,,,,
04-03-08,4864.250000,12213.799805,1326.750000,2260.280029,97.519997,3.579,963.900024,40.209000,73.660004,4852.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05-03-08,4921.399902,12254.990234,1333.699951,2272.810059,101.639999,3.693,986.200012,40.174000,73.480003,4915.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-03-08,4771.600098,11893.690430,1293.369995,2212.489990,102.379997,3.541,972.200012,40.238998,73.029999,4706.0,...,NaN,NaN,-4.274937,NaN,NaN,NaN,-0.870647,NaN,NaN,NaN
10-03-08,4800.399902,11740.150391,1273.369995,2169.340088,104.160004,3.438,969.900024,40.313999,72.989998,4706.0,...,NaN,NaN,-1.000000,NaN,NaN,NaN,7.526045,NaN,NaN,NaN
11-03-08,4865.899902,12156.809570,1320.650024,2255.760010,105.250000,3.596,974.200012,40.174999,73.279999,4846.5,...,NaN,NaN,inf,NaN,NaN,NaN,-1.045361,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29-03-23,17080.699219,32717.599609,4027.810059,11926.240234,78.279999,3.566,1966.099976,82.164299,102.639999,17200.0,...,-1.839739,0.030582,-3.567208,-0.309082,-0.435016,-1.098680,3.365632,-0.127033,-0.865636,-56.521208
31-03-23,17359.750000,33274.148438,4109.310059,12221.910156,79.769997,3.494,1969.000000,82.100502,102.510002,17480.5,...,-1.808875,-0.215949,0.322594,-6.471258,-0.830032,-0.995031,-0.478895,0.007460,-0.979919,89.076748
03-04-23,17398.050781,33601.148438,4124.509766,12189.450195,84.930000,3.430,1983.900024,82.184799,102.089996,17456.5,...,-1.743663,-0.428063,-1.084189,-56.891259,-0.967791,-1.085774,-0.467526,-6.444835,-0.869266,3.085447


In [13]:
df1.columns

Index(['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y',
       'Gold', 'Inr_Usd', 'Dollar_Index', 'sgx', 'indiavix',
       'Nifty_50_1d_return', 'Nifty_50_7d_return', 'Nifty_50_1m_return',
       'Nifty_50_1y_return', 'Dow_Jons_1d_return', 'Dow_Jons_7d_return',
       'Dow_Jons_1m_return', 'Dow_Jons_1y_return', 'S&P_500_1d_return',
       'S&P_500_7d_return', 'S&P_500_1m_return', 'S&P_500_1y_return',
       'NASDAQ_1d_return', 'NASDAQ_7d_return', 'NASDAQ_1m_return',
       'NASDAQ_1y_return', 'BRENT_1d_return', 'BRENT_7d_return',
       'BRENT_1m_return', 'BRENT_1y_return', 'BOND_10Y_1d_return',
       'BOND_10Y_7d_return', 'BOND_10Y_1m_return', 'BOND_10Y_1y_return',
       'Gold_1d_return', 'Gold_7d_return', 'Gold_1m_return', 'Gold_1y_return',
       'Inr_Usd_1d_return', 'Inr_Usd_7d_return', 'Inr_Usd_1m_return',
       'Inr_Usd_1y_return', 'Dollar_Index_1d_return', 'Dollar_Index_7d_return',
       'Dollar_Index_1m_return', 'Dollar_Index_1y_return', 'sgx_1d_return',
 

In [14]:
df3 = df1.drop(columns=['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y',
       'Gold', 'Inr_Usd', 'Dollar_Index', 'sgx', 'indiavix',])
# Fill all NaN values with 0
#df3.fillna(0, inplace=True)
df3 = df3.round(2)
df3

,Nifty_50_1d_return,Nifty_50_7d_return,Nifty_50_1m_return,Nifty_50_1y_return,Dow_Jons_1d_return,Dow_Jons_7d_return,Dow_Jons_1m_return,Dow_Jons_1y_return,S&P_500_1d_return,S&P_500_7d_return,...,Dollar_Index_1m_return,Dollar_Index_1y_return,sgx_1d_return,sgx_7d_return,sgx_1m_return,sgx_1y_return,indiavix_1d_return,indiavix_7d_return,indiavix_1m_return,indiavix_1y_return
Date,,,,,,,,,,,,,,,,,,,,,
04-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-03-08,-3.59,NaN,NaN,NaN,-9.74,NaN,NaN,NaN,-6.77,NaN,...,NaN,NaN,-4.27,NaN,NaN,NaN,-0.87,NaN,NaN,NaN
10-03-08,-1.20,NaN,NaN,NaN,-0.56,NaN,NaN,NaN,-0.49,NaN,...,NaN,NaN,-1.00,NaN,NaN,NaN,7.53,NaN,NaN,NaN
11-03-08,1.26,NaN,NaN,NaN,-3.75,NaN,NaN,NaN,-3.40,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,-1.05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29-03-23,-4.80,-1.39,-0.34,-1.13,-9.56,3.40,-1.06,-2.09,-10.05,0.99,...,-1.84,0.03,-3.57,-0.31,-0.44,-1.10,3.37,-0.13,-0.87,-56.52
31-03-23,1.15,2.08,-0.56,-1.03,0.70,-9.31,-1.45,-1.79,0.42,-14.93,...,-1.81,-0.22,0.32,-6.47,-0.83,-1.00,-0.48,0.01,-0.98,89.08
03-04-23,-0.86,-3.95,-1.26,-1.08,-0.42,2.59,-1.57,-1.70,-0.82,1.79,...,-1.74,-0.43,-1.08,-56.89,-0.97,-1.09,-0.47,-6.44,-0.87,3.09


In [15]:
df2

,us_unemployment,US_CPI,FED_RATE,india_cpi,bond_Return
DATE,,,,,
01-06-07,4.6,2.687038,5.25,5.691057,0.00
01-07-07,4.7,2.358231,5.26,6.451613,0.00
01-08-07,4.6,1.970083,5.02,7.258065,0.00
01-09-07,4.7,2.755052,4.94,6.400000,0.00
01-10-07,4.7,3.536174,4.76,5.511811,0.00
...,...,...,...,...,...
01-10-22,3.7,7.745427,3.08,6.084868,7.46
01-11-22,3.6,7.110323,3.78,5.409706,7.36
01-12-22,3.5,6.454401,4.10,5.502392,7.29


In [16]:
for col in df2.columns:
    df2[f'{col}_1m_return'] = df2[col].pct_change(periods=1)
    df2[f'{col}_1m_return'] = df2[f'{col}_1m_return'].pct_change(periods=1)

    df2[f'{col}_1y_return'] = df2[col].pct_change(periods=12)
    df2[f'{col}_1y_return'] = df2[f'{col}_1y_return'].pct_change(periods=12)
df2

,us_unemployment,US_CPI,FED_RATE,india_cpi,bond_Return,us_unemployment_1m_return,us_unemployment_1y_return,US_CPI_1m_return,US_CPI_1y_return,FED_RATE_1m_return,FED_RATE_1y_return,india_cpi_1m_return,india_cpi_1y_return,bond_Return_1m_return,bond_Return_1y_return
DATE,,,,,,,,,,,,,,,
01-06-07,4.6,2.687038,5.25,5.691057,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-07-07,4.7,2.358231,5.26,6.451613,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-08-07,4.6,1.970083,5.02,7.258065,0.00,-1.978723,NaN,0.345065,NaN,-24.954373,NaN,-0.064655,NaN,NaN,NaN
01-09-07,4.7,2.755052,4.94,6.400000,0.00,-2.021739,NaN,-3.420789,NaN,-0.650730,NaN,-1.945778,NaN,NaN,NaN
01-10-07,4.7,3.536174,4.76,5.511811,0.00,-1.000000,NaN,-0.288423,NaN,1.286437,NaN,0.173887,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01-10-22,3.7,7.745427,3.08,6.084868,7.46,-2.057143,-0.488889,6.532208,-0.942566,1.057745,-338.500,-1.573481,-2.478635,inf,1.509971
01-11-22,3.6,7.110323,3.78,5.409706,7.36,-1.472973,-0.617143,0.474031,-0.990775,0.118881,-417.250,0.784752,-2.397513,-1.542151,1.036597
01-12-22,3.5,6.454401,4.10,5.502392,7.29,0.027778,-0.754579,0.125026,-1.019853,-0.627513,-453.250,-1.154414,-1.018877,-0.290489,0.413467


In [17]:
df2.columns

Index(['us_unemployment', 'US_CPI', 'FED_RATE', 'india_cpi', 'bond_Return',
       'us_unemployment_1m_return', 'us_unemployment_1y_return',
       'US_CPI_1m_return', 'US_CPI_1y_return', 'FED_RATE_1m_return',
       'FED_RATE_1y_return', 'india_cpi_1m_return', 'india_cpi_1y_return',
       'bond_Return_1m_return', 'bond_Return_1y_return'],
      dtype='object')

In [18]:
df4 = df2.drop(columns=['us_unemployment', 'US_CPI', 'FED_RATE', 'india_cpi', 'bond_Return','us_unemployment_1y_return',
       'US_CPI_1y_return','FED_RATE_1y_return','india_cpi_1y_return','bond_Return_1m_return', 'bond_Return_1y_return'])
df5 = df1.drop(columns=['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y','Gold', 'Inr_Usd', 'Dollar_Index',
                        'sgx', 'indiavix','Nifty_50_1d_return', 'Nifty_50_7d_return', 'Nifty_50_1y_return', 'Dow_Jons_1d_return',
                        'Dow_Jons_7d_return','Dow_Jons_1y_return', 'S&P_500_1d_return','S&P_500_7d_return', 'S&P_500_1m_return',
                        'S&P_500_1y_return','NASDAQ_1d_return', 'NASDAQ_7d_return', 'NASDAQ_1y_return', 'BRENT_1d_return', 'BRENT_7d_return',
                        'BRENT_1y_return', 'BOND_10Y_1d_return','BOND_10Y_7d_return', 'BOND_10Y_1y_return','Gold_1d_return', 'Gold_7d_return',
                        'Gold_1y_return','Inr_Usd_1d_return', 'Inr_Usd_7d_return', 'Inr_Usd_1y_return', 'Dollar_Index_1d_return', 'Dollar_Index_7d_return',
                        'Dollar_Index_1y_return', 'sgx_1d_return','sgx_7d_return','sgx_1y_return', 'indiavix_1d_return','indiavix_7d_return','indiavix_1y_return'])
# Assuming your DataFrame is called df5
df5.index = pd.to_datetime(df5.index)
df5 = df5.resample('MS').mean() # Use 'MS' instead of 'M'
df5.index = df5.index.strftime('%d-%m-%y')
# Fill all NaN values with 0
df4.fillna(0, inplace=True)
df4 = df4.round(2)

In [19]:
df6 = df2.drop(columns=['us_unemployment', 'US_CPI', 'FED_RATE', 'india_cpi', 'bond_Return','us_unemployment_1m_return',
                        'US_CPI_1m_return', 'US_CPI_1y_return', 'FED_RATE_1m_return', 'india_cpi_1m_return', 'bond_Return_1m_return', ])
df7 = df1.drop(columns=['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y','Gold', 'Inr_Usd', 'Dollar_Index',
                        'sgx', 'indiavix','Nifty_50_1d_return', 'Nifty_50_7d_return', 'Nifty_50_1m_return', 'Dow_Jons_1d_return',
                        'Dow_Jons_7d_return','Dow_Jons_1m_return', 'S&P_500_1d_return','S&P_500_7d_return', 'S&P_500_1m_return',
                        'NASDAQ_1d_return', 'NASDAQ_7d_return', 'NASDAQ_1m_return','BRENT_1d_return', 'BRENT_7d_return',
                        'BRENT_1m_return', 'BOND_10Y_1d_return','BOND_10Y_7d_return', 'BOND_10Y_1m_return', 'Gold_1d_return',
                        'Gold_7d_return', 'Gold_1m_return', 'Inr_Usd_1d_return', 'Inr_Usd_7d_return', 'Inr_Usd_1m_return',
                        'Dollar_Index_1d_return', 'Dollar_Index_7d_return','Dollar_Index_1m_return', 'sgx_1d_return',
                        'sgx_7d_return', 'sgx_1m_return',  'indiavix_1d_return','indiavix_7d_return', 'indiavix_1m_return'])
# Assuming your DataFrame is called df5
df7.index = pd.to_datetime(df7.index)
df7 = df7.resample('MS').mean() # Use 'MS' instead of 'M'
df7.index = df7.index.strftime('%d-%m-%y')

In [20]:
df7

,Nifty_50_1y_return,Dow_Jons_1y_return,S&P_500_1y_return,NASDAQ_1y_return,BRENT_1y_return,BOND_10Y_1y_return,Gold_1y_return,Inr_Usd_1y_return,Dollar_Index_1y_return,sgx_1y_return,indiavix_1y_return
Date,,,,,,,,,,,
01-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
01-08-23,-0.121773,-1.093908,-1.538799,-1.499851,-1.272302,2.381361,-6.082945,1.703101,0.217010,0.012133,-2.515521
01-09-23,-0.653418,-1.508377,-1.690820,-4.414715,-1.209471,1.490911,-104.614127,3.327941,0.274746,-0.668437,-1.777841
01-10-23,-0.812022,-1.545517,-1.758084,-17.283698,-1.175450,0.847132,-3.855474,4.121093,0.165251,-0.804170,-1.512007


In [21]:
df6

,us_unemployment_1y_return,FED_RATE_1y_return,india_cpi_1y_return,bond_Return_1y_return
DATE,,,,
01-06-07,NaN,NaN,NaN,NaN
01-07-07,NaN,NaN,NaN,NaN
01-08-07,NaN,NaN,NaN,NaN
01-09-07,NaN,NaN,NaN,NaN
01-10-07,NaN,NaN,NaN,NaN
...,...,...,...,...
01-10-22,-0.488889,-338.500,-2.478635,1.509971
01-11-22,-0.617143,-417.250,-2.397513,1.036597
01-12-22,-0.754579,-453.250,-1.018877,0.413467


In [22]:
return_1y = pd.concat([df6,df7], axis=1)
return_1y = return_1y.dropna()
return_1y.replace([np.inf, -np.inf], 0, inplace=True)
return_1y

,us_unemployment_1y_return,FED_RATE_1y_return,india_cpi_1y_return,bond_Return_1y_return,Nifty_50_1y_return,Dow_Jons_1y_return,S&P_500_1y_return,NASDAQ_1y_return,BRENT_1y_return,BOND_10Y_1y_return,Gold_1y_return,Inr_Usd_1y_return,Dollar_Index_1y_return,sgx_1y_return,indiavix_1y_return
01-12-12,-0.179412,-3.103896,-3.291962,-1.000000,-1.903389,4.032268,-2.338249,-2.180957,-0.898558,2.169569,-0.888880,-3.430186,-0.434921,-1.901485,-2.116807
01-01-13,-0.588855,-2.416667,-3.764741,-1.000000,-1.864414,3.320984,1.461299,-7.332898,-1.126153,0.172977,-2.030506,-0.581497,-1.035518,-1.873055,-2.892508
01-02-13,-0.070568,-2.333333,-5.170809,-1.000000,-3.980358,5.374780,0.263311,1.923878,-1.351174,-1.302716,-2.011685,-0.161199,-2.407494,19.474384,-3.409388
01-03-13,-0.039634,-2.076923,-17.300458,-1.000000,-10.537118,2.012143,8.406857,2.592086,-2.665486,-1.394923,1.388071,-0.096820,1.803683,-0.025765,-5.018268
01-04-13,-0.260163,-0.821429,-0.966911,-1.000000,-2.847341,31.786410,0.563889,4.504704,-2.414882,-1.018429,1.748531,-0.392238,-0.332620,-2.871471,1.023482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01-10-22,-0.488889,-338.500000,-2.478635,1.509971,-0.903734,-1.188335,-1.292761,-1.702867,-0.786077,-0.041580,0.047239,0.077939,-5.877381,-0.899858,-1.242082
01-11-22,-0.617143,-417.250000,-2.397513,1.036597,-0.775301,-0.999641,-1.226113,-1.638231,-0.656647,0.188392,4.189681,4.445757,0.779283,-0.770745,-14.909857
01-12-22,-0.754579,-453.250000,-1.018877,0.413467,-0.722503,-1.224599,-1.423753,-2.053319,-0.966960,0.952676,2.002440,0.545825,-1.749303,-0.723547,0.556743
01-01-23,-0.589130,-479.125000,-0.935677,-0.069025,-0.895511,-1.266339,-1.599879,-5.055466,-1.068463,0.661038,-5.105408,4.107948,0.060153,-0.876654,-1.888587


In [23]:
return_1m = pd.concat([df4,df5], axis=1)
return_1m = return_1m.dropna()
return_1m.replace([np.inf, -np.inf], 0, inplace=True)
return_1m

,us_unemployment_1m_return,US_CPI_1m_return,FED_RATE_1m_return,india_cpi_1m_return,Nifty_50_1m_return,Dow_Jons_1m_return,NASDAQ_1m_return,BRENT_1m_return,BOND_10Y_1m_return,Gold_1m_return,Inr_Usd_1m_return,Dollar_Index_1m_return,sgx_1m_return,indiavix_1m_return
01-01-08,-1.00,-1.93,0.27,0.00,-1.740273,-1.907285,-2.148511,-0.389611,-49.147757,-1.427281,0.064940,-0.790888,-1.667327,-2.658513
01-02-08,0.00,-2.22,2.44,0.00,-1.364678,-0.573348,-17.204443,-0.536691,-2.640800,-1.066902,0.853322,-0.849704,-1.514928,-1.069338
01-03-08,-3.04,-0.81,-0.49,-57.30,-0.622559,-2.378893,-5.037357,0.907588,-2.098772,-0.377219,0.242068,3.969377,-0.500891,2.568136
01-04-08,-1.48,-0.00,0.02,-1.02,-0.916326,-0.657176,-0.308142,-1.017062,3.456409,-2.192712,1.965266,-1.727833,-0.910453,-0.403968
01-05-08,-5.08,-6.42,0.04,-0.01,0.490591,-0.665432,0.354165,0.301586,0.212544,-0.967394,-1.428208,0.611462,0.800141,-1.184897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01-10-22,-2.06,6.53,1.06,-1.57,-3.249616,-1.418500,-5.992134,-1.228532,-0.233398,-0.589414,-14.559356,-2.117325,-4.329774,-1.998233
01-11-22,-1.47,0.47,0.12,0.78,-2.322544,-0.514827,0.170210,-1.664327,-0.511814,0.029025,0.270736,-1.808025,-2.044104,1.517237
01-12-22,0.03,0.13,-0.63,-1.15,-1.035931,-0.737765,-2.483332,0.743761,-0.569696,-0.808562,-1.461215,-0.787486,-1.078462,-1.367788
01-01-23,0.03,-0.93,-0.34,5.92,-6.263978,-1.718143,-1.998147,0.033959,-1.335379,3.679154,-2.045545,-0.080379,-1.001341,-1.132070


In [24]:
df3.columns

Index(['Nifty_50_1d_return', 'Nifty_50_7d_return', 'Nifty_50_1m_return',
       'Nifty_50_1y_return', 'Dow_Jons_1d_return', 'Dow_Jons_7d_return',
       'Dow_Jons_1m_return', 'Dow_Jons_1y_return', 'S&P_500_1d_return',
       'S&P_500_7d_return', 'S&P_500_1m_return', 'S&P_500_1y_return',
       'NASDAQ_1d_return', 'NASDAQ_7d_return', 'NASDAQ_1m_return',
       'NASDAQ_1y_return', 'BRENT_1d_return', 'BRENT_7d_return',
       'BRENT_1m_return', 'BRENT_1y_return', 'BOND_10Y_1d_return',
       'BOND_10Y_7d_return', 'BOND_10Y_1m_return', 'BOND_10Y_1y_return',
       'Gold_1d_return', 'Gold_7d_return', 'Gold_1m_return', 'Gold_1y_return',
       'Inr_Usd_1d_return', 'Inr_Usd_7d_return', 'Inr_Usd_1m_return',
       'Inr_Usd_1y_return', 'Dollar_Index_1d_return', 'Dollar_Index_7d_return',
       'Dollar_Index_1m_return', 'Dollar_Index_1y_return', 'sgx_1d_return',
       'sgx_7d_return', 'sgx_1m_return', 'sgx_1y_return', 'indiavix_1d_return',
       'indiavix_7d_return', 'indiavix_1m_return', 'ind

In [25]:
return_1d = df1.drop(columns=['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y','Gold', 'Inr_Usd', 'Dollar_Index',
                              'sgx', 'indiavix','Nifty_50_7d_return', 'Nifty_50_1m_return','Nifty_50_1y_return', 'Dow_Jons_7d_return',
                              'Dow_Jons_1m_return', 'Dow_Jons_1y_return', 'S&P_500_7d_return', 'S&P_500_1m_return', 'S&P_500_1y_return',
                              'NASDAQ_7d_return', 'NASDAQ_1m_return','NASDAQ_1y_return', 'BRENT_7d_return', 'BRENT_1m_return', 'BRENT_1y_return',
                              'BOND_10Y_7d_return', 'BOND_10Y_1m_return', 'BOND_10Y_1y_return','Gold_7d_return', 'Gold_1m_return', 'Gold_1y_return',
                              'Inr_Usd_7d_return', 'Inr_Usd_1m_return','Inr_Usd_1y_return', 'Dollar_Index_7d_return','Dollar_Index_1m_return',
                              'Dollar_Index_1y_return',  'sgx_7d_return', 'sgx_1m_return', 'sgx_1y_return', 'indiavix_7d_return', 'indiavix_1m_return',
                              'indiavix_1y_return'])
return_1d.replace([np.inf, -np.inf], np.nan, inplace=True)
return_1d = return_1d.dropna()
return_1d

,Nifty_50_1d_return,Dow_Jons_1d_return,S&P_500_1d_return,NASDAQ_1d_return,BRENT_1d_return,BOND_10Y_1d_return,Gold_1d_return,Inr_Usd_1d_return,Dollar_Index_1d_return,sgx_1d_return,indiavix_1d_return
Date,,,,,,,,,,,
07-03-08,-3.590735,-9.741968,-6.772673,-5.787501,-0.827670,-2.292175,-1.613607,-2.858730,1.506145,-4.274937,-0.870647
10-03-08,-1.198291,-0.562125,-0.488627,-0.265148,1.388035,-0.293282,-0.833349,0.152018,-0.910562,-1.000000,7.526045
12-03-08,-0.908122,-1.107938,-1.242273,-1.132312,-0.073911,-1.683766,0.065043,0.414945,-4.022455,-0.892878,-3.989905
13-03-08,-41.669658,-1.765236,-1.569938,-2.669030,0.233156,-1.465969,1.921006,-1.743428,-0.620439,-17.270544,-6.479558
14-03-08,-1.518377,-6.467083,-5.053751,-3.567089,-2.042657,-3.183715,-0.568907,-1.336707,0.248094,-1.602498,-2.609305
...,...,...,...,...,...,...,...,...,...,...,...
29-03-23,-4.801728,-9.557403,-10.046178,-5.001279,-1.693416,-0.945003,-1.311808,-0.591010,-1.490420,-3.567208,3.365632
31-03-23,1.146847,0.704188,0.421222,0.382088,-5.046026,-36.978140,-1.461793,0.261633,-1.617771,0.322594,-0.478895
03-04-23,-0.864952,-0.422278,-0.817199,-1.107129,2.398409,-0.092796,4.130329,-2.322361,2.234979,-1.084189,-0.467526


In [26]:
return_1w = df1.drop(columns=['Nifty_50', 'Dow_Jons', 'S&P_500', 'NASDAQ', 'BRENT', 'BOND_10Y','Gold', 'Inr_Usd', 'Dollar_Index', 'sgx',
                              'indiavix','Nifty_50_1d_return', 'Nifty_50_1m_return','Nifty_50_1y_return', 'Dow_Jons_1d_return', 'Dow_Jons_1m_return',
                              'Dow_Jons_1y_return', 'S&P_500_1d_return','S&P_500_1m_return', 'S&P_500_1y_return','NASDAQ_1d_return',  'NASDAQ_1m_return',
                              'NASDAQ_1y_return', 'BRENT_1d_return', 'BRENT_1m_return', 'BRENT_1y_return', 'BOND_10Y_1d_return', 'BOND_10Y_1m_return',
                              'BOND_10Y_1y_return','Gold_1d_return', 'Gold_1m_return', 'Gold_1y_return','Inr_Usd_1d_return',  'Inr_Usd_1m_return',
                              'Inr_Usd_1y_return', 'Dollar_Index_1d_return', 'Dollar_Index_1m_return', 'Dollar_Index_1y_return', 'sgx_1d_return',
                              'sgx_1m_return', 'sgx_1y_return', 'indiavix_1d_return','indiavix_1m_return', 'indiavix_1y_return'])
return_1w.replace([np.inf, -np.inf], np.nan, inplace=True)
return_1w = return_1w.dropna()
return_1w

,Nifty_50_7d_return,Dow_Jons_7d_return,S&P_500_7d_return,NASDAQ_7d_return,BRENT_7d_return,BOND_10Y_7d_return,Gold_7d_return,Inr_Usd_7d_return,Dollar_Index_7d_return,sgx_7d_return,indiavix_7d_return
Date,,,,,,,,,,,
19-03-08,-39.396863,-0.896962,-0.416453,1.081556,-1.582060,0.295158,-3.253756,-1.673416,-0.790058,-27.944467,-1.643927
24-03-08,-0.950854,-4.721039,-2.914190,-7.890962,-2.230279,-0.921134,-13.056622,-2.221555,-1.636320,-1.121973,-1.810451
26-03-08,-2.168034,0.903815,18.927910,18.143736,-1.951473,-2.505928,-2.611166,-6.275383,-1.033381,-2.615742,3.299590
27-03-08,-1.958486,-1.375168,-1.489815,0.000064,-2.801153,-1.596464,-2.821636,8.957612,-1.053891,-2.008395,-1.093939
28-03-08,-2.315327,-12.043615,-2.636124,-2.533638,-1.579831,-1.890439,-0.571583,-4.350729,0.775589,-2.547021,-1.704975
...,...,...,...,...,...,...,...,...,...,...,...
29-03-23,-1.391961,3.398585,0.993800,0.064386,-0.490734,7.231247,-0.054475,-2.371659,-1.128921,-0.309082,-0.127033
31-03-23,2.081434,-9.308821,-14.932475,5.159787,2.139210,-1.517457,-1.319113,1.244808,-0.989225,-6.471258,0.007460
03-04-23,-3.949314,2.588704,1.785985,0.858414,3.788250,-4.348153,-0.854562,-0.773857,0.755773,-56.891259,-6.444835


In [27]:
with open('return_1d.pkl', 'wb') as fh:
   pickle.dump(return_1d, fh)

In [28]:
with open('return_1w.pkl', 'wb') as fh:
   pickle.dump(return_1w, fh)
with open('return_1m.pkl', 'wb') as fh:
   pickle.dump(return_1m, fh)
with open('return_1y.pkl', 'wb') as fh:
   pickle.dump(return_1y, fh)